# Clasificador de texto para el verificador de noticias (RealCheck)

En este proyecto, se implementará un clasificador de texto utilizando técnicas de aprendizaje automático con el objetivo de verificar si la frase u oración proporcionada puede corresponder a una noticia verificable o por el contrario, se trata de un comentario o pregunta que no tiene una base factual clara. 

El modelo entrenado será utilizado en la producción para predecir la verificabilidad de las noticias. 

## Instalación de los paquetes necesarios

In [ ]:
!pip3 install torch
!pip3 install spacy
!python3 -m spacy download es_dep_news_trf

In [2]:
!pip3 install --upgrade scikit-learn

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 9.8 MB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 297 kB 1.4 MB/s eta 0:00:01


## Importación de los módulos a usar

In [15]:
import pandas as pd
import spacy
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

## Cargar los datos del archivo CSV

In [5]:
df = pd.read_csv('Dataset.csv', header=None, names=['texto', 'noticia'])
print(df)

print(df["noticia"].value_counts())

                                                  texto  noticia
0     No puedo creer que haya personas que sigan jus...        0
1           Hoy hace un día precioso para ir a la playa        0
2     Encuentran en San Andrés a migrantes que iban ...        1
3     Polémica por tarifas diferenciales a extranjer...        1
4     Se canceló el partido de fútbol de este fin de...        0
...                                                 ...      ...
1995  Nevado del Ruiz: ¿cuáles son los planes de con...        1
1996  Hoy voy a hacer una llamada telefónica a un am...        0
1997  Por ahora, Colombia no firmará nuevos contrato...        1
1998  Hipopótamo causa otro accidente en Puerto Triu...        1
1999  Me parece increíble que haya personas que toda...        0

[2000 rows x 2 columns]
1    1000
0    1000
Name: noticia, dtype: int64


## Preprocesar los datos

In [6]:
nlp = spacy.load("es_dep_news_trf")

def Tokenizar(text):
    tokens = [token.lemma_.lower() for token in nlp(text) if not token.is_stop and not token.is_punct]
    return tokens

In [11]:
# Mostrar ejemplo de preprocesamiento
print(Tokenizar(df['texto'][0]))
print(Tokenizar(df['texto'][5]))

['creer', 'persona', 'seguir', 'justificar', 'discriminación', 'racismo', 'pleno', 'siglo', 'xxi']
['abuela', 'galleta', 'avena', 'visitar']


## Separar los datos en conjuntos de entrenamiento y prueba

In [12]:
X = df['texto']
y = df['noticia']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=df["noticia"])

print("Cantidad datos entrenamiento: ",len(X_train))
print("Cantidad datos prueba: ",len(X_test))

Cantidad datos entrenamiento:  1600
Cantidad datos prueba:  400


## Vectorizar

In [13]:
real_vectorizer = CountVectorizer(tokenizer = Tokenizar, binary=True,  token_pattern=None)
train_X = real_vectorizer.fit_transform(X_train)
test_X = real_vectorizer.transform(X_test)

In [16]:
clf = LogisticRegression(C=1, penalty='l2', solver='liblinear')
clf.fit(train_X, y_train)

# Hacer predicciones con los datos de prueba
y_pred = clf.predict(test_X)

# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calcular la precisión del modelo
precision = precision_score(y_test, y_pred)
print("Precision:", precision)

# Calcular la sensibilidad del modelo
recall = recall_score(y_test, y_pred)
print("Recall:", recall)

# Calcular la especificidad del modelo
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
print("Specificity:", specificity)

Accuracy: 0.92
Precision: 0.9666666666666667
Recall: 0.87
Specificity: 0.97


In [20]:
frases = [
  'Al perro le gusta comer carne en el centro de manhatan',
  'Nuevo informe muestra el aumento de la violencia de género en la región.',
  'Empresa anuncia el lanzamiento de un nuevo producto revolucionario en el mercado.',
  'La pizza no me gusta',
  'Manuela le gusta cepillarse todo los dias en el congreso',
  'La marihuana queda totalmente prohibida en toda Colombia'
]

frases_X = real_vectorizer.transform(frases)
predicciones = clf.predict(frases_X)
probabilidades = clf.predict_proba(frases_X)


for text, label, p in zip(frases, predicciones, probabilidades):
    if label == 0:
        print(f"{label} - {text}: Probabilidad: {round(p[0],2)}")
    else:
        print(f"{label} - {text}: Probabilidad: {round(p[1],2)}")

0 - Al perro le gusta comer carne en el centro de manhatan: Probabilidad: 0.97
1 - Nuevo informe muestra el aumento de la violencia de género en la región.: Probabilidad: 0.68
1 - Empresa anuncia el lanzamiento de un nuevo producto revolucionario en el mercado.: Probabilidad: 0.91
0 - La pizza no me gusta: Probabilidad: 0.89
0 - Manuela le gusta cepillarse todo los dias en el congreso: Probabilidad: 0.85
1 - La marihuana queda totalmente prohibida en toda Colombia: Probabilidad: 0.94


In [19]:
import joblib

# Exportar el vectorizador
joblib.dump(real_vectorizer, 'real_vectorizer.joblib')

# Exportar el modelo
joblib.dump(clf, 'clf.joblib')


['clf.joblib']